In [1]:
from ds_utils.ds_preamble import *
from ds_utils.ds_plotting import *
from ds_utils.ds_helper import *
import datetime
import duckdb

calling ds_preamble..
calling ds_helper...


In [2]:
tips = sns.load_dataset('tips')
titanic = sns.load_dataset('titanic')
# tips.sort_index(axis=1)
# tips.reindex(sorted(tips.columns), axis=1)

# tips datasets

In [3]:
tips.info(memory_usage='deep')
tips.sex.dtype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.9 KB


CategoricalDtype(categories=['Male', 'Female'], ordered=False)

In [55]:
# ------------------------ append the global mean tip ------------------------ #
tips.assign(avg_tip=tips.tip.mean())

q = """
SELECT *, avg(tip) over() AS avg_tip FROM tips
"""

# ------------------- append a column of mean tip by cohort ------------------ #
tips.assign(avg_tip=tips.groupby(['sex','smoker']).tip.transform('mean'))

q = """
SELECT *, avg(tip) over(PARTITION BY sex, smoker) AS avg_tip FROM tips
"""

# ----------------- append several columns of mean by cohort ----------------- #
tips[['avg_tip', 'avg_total_bill']] = tips.groupby(['sex','smoker'])[['tip','total_bill']].transform('mean')
tips

# M1: use window function
q ="""
SELECT *, 
avg(tip) over(PARTITION BY sex, smoker) AS avg_tip, 
avg(total_bill) OVER (PARTITION BY sex, smoker) AS avg_total_bill 
FROM tips
"""

# M2: use join 
q = """
WITH TEMP AS (
    SELECT sex, smoker, avg(tip) AS avg_tip, avg(total_bill) AS avg_total_bill FROM tips GROUP BY sex, smoker
    ) 
SELECT * FROM tips NATURAL JOIN TEMP 
"""


,total_bill,tip,sex,smoker,day,time,size,avg_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,2.998279
1,10.34,1.66,Male,No,Sun,Dinner,3,2.998279
2,21.01,3.50,Male,No,Sun,Dinner,3,2.998279
3,23.68,3.31,Male,No,Sun,Dinner,2,2.998279
4,24.59,3.61,Female,No,Sun,Dinner,4,2.998279
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,2.998279
240,27.18,2.00,Female,Yes,Sat,Dinner,2,2.998279
241,22.67,2.00,Male,Yes,Sat,Dinner,2,2.998279
242,17.82,1.75,Male,No,Sat,Dinner,2,2.998279


,total_bill,tip,sex,smoker,day,time,size,avg_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,2.773519
1,10.34,1.66,Male,No,Sun,Dinner,3,3.113402
2,21.01,3.50,Male,No,Sun,Dinner,3,3.113402
3,23.68,3.31,Male,No,Sun,Dinner,2,3.113402
4,24.59,3.61,Female,No,Sun,Dinner,4,2.773519
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,3.113402
240,27.18,2.00,Female,Yes,Sat,Dinner,2,2.931515
241,22.67,2.00,Male,Yes,Sat,Dinner,2,3.051167
242,17.82,1.75,Male,No,Sat,Dinner,2,3.113402


,total_bill,tip,sex,smoker,day,time,size,avg_tip,avg_total_bill
0,16.99,1.01,Female,No,Sun,Dinner,2,2.773519,18.105185
1,10.34,1.66,Male,No,Sun,Dinner,3,3.113402,19.791237
2,21.01,3.50,Male,No,Sun,Dinner,3,3.113402,19.791237
3,23.68,3.31,Male,No,Sun,Dinner,2,3.113402,19.791237
4,24.59,3.61,Female,No,Sun,Dinner,4,2.773519,18.105185
...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,3.113402,19.791237
240,27.18,2.00,Female,Yes,Sat,Dinner,2,2.931515,17.977879
241,22.67,2.00,Male,Yes,Sat,Dinner,2,3.051167,22.284500
242,17.82,1.75,Male,No,Sat,Dinner,2,3.113402,19.791237


In [69]:
# ----------------------- find the ranking based on global tip ---------------------- # 
tips.assign(rank_tip=tips.tip.rank(method='min')).sort_values('tip')
q = """
SELECT *, rank() over(ORDER BY tip) FROM tips ORDER BY tip
"""

# ----------------------- find the ranking based on tip by cohort ---------------------- # 
tips.assign(rank_tip=tips.groupby(['sex','smoker']).tip.rank(method='min')).sort_values('tip')
q = """
SELECT *, rank() over(PARTITION BY sex, smoker ORDER BY tip) FROM tips ORDER BY tip
"""

,total_bill,tip,sex,smoker,day,time,size,avg_tip,avg_total_bill,rank_tip
67,3.07,1.00,Female,Yes,Sat,Dinner,1,2.931515,17.977879,1.0
236,12.60,1.00,Male,Yes,Sat,Dinner,2,3.051167,22.284500,1.0
92,5.75,1.00,Female,Yes,Fri,Dinner,2,2.931515,17.977879,1.0
111,7.25,1.00,Female,No,Sat,Dinner,1,2.773519,18.105185,1.0
0,16.99,1.01,Female,No,Sun,Dinner,2,2.773519,18.105185,5.0
...,...,...,...,...,...,...,...,...,...,...
141,34.30,6.70,Male,No,Thur,Lunch,6,3.113402,19.791237,240.0
59,48.27,6.73,Male,No,Sat,Dinner,4,3.113402,19.791237,241.0
23,39.42,7.58,Male,No,Sat,Dinner,4,3.113402,19.791237,242.0
212,48.33,9.00,Male,No,Sat,Dinner,4,3.113402,19.791237,243.0


,total_bill,tip,sex,smoker,day,time,size,avg_tip,avg_total_bill,rank_tip
67,3.07,1.00,Female,Yes,Sat,Dinner,1,2.931515,17.977879,1.0
236,12.60,1.00,Male,Yes,Sat,Dinner,2,3.051167,22.284500,1.0
92,5.75,1.00,Female,Yes,Fri,Dinner,2,2.931515,17.977879,1.0
111,7.25,1.00,Female,No,Sat,Dinner,1,2.773519,18.105185,1.0
0,16.99,1.01,Female,No,Sun,Dinner,2,2.773519,18.105185,2.0
...,...,...,...,...,...,...,...,...,...,...
141,34.30,6.70,Male,No,Thur,Lunch,6,3.113402,19.791237,94.0
59,48.27,6.73,Male,No,Sat,Dinner,4,3.113402,19.791237,95.0
23,39.42,7.58,Male,No,Sat,Dinner,4,3.113402,19.791237,96.0
212,48.33,9.00,Male,No,Sat,Dinner,4,3.113402,19.791237,97.0


,total_bill,tip,sex,smoker,day,time,size,avg_tip
0,16.99,1.01,Female,No,Sun,Dinner,2,2.998279
1,10.34,1.66,Male,No,Sun,Dinner,3,2.998279
2,21.01,3.50,Male,No,Sun,Dinner,3,2.998279
3,23.68,3.31,Male,No,Sun,Dinner,2,2.998279
4,24.59,3.61,Female,No,Sun,Dinner,4,2.998279
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,2.998279
240,27.18,2.00,Female,Yes,Sat,Dinner,2,2.998279
241,22.67,2.00,Male,Yes,Sat,Dinner,2,2.998279
242,17.82,1.75,Male,No,Sat,Dinner,2,2.998279


In [4]:
(
tips # from 
.query("size >= 3") # where 
.groupby(['smoker','sex']) # group by 
.filter(lambda g: len(g)>10 and g.tip.mean()>3) # having 
.groupby(['smoker','sex'], observed=True) # group by again...
.total_bill.mean() # select 
.sort_values(ascending=False) # order by DESC 
.head(2) # limit 
)

q = """
SELECT smoker, sex, avg(total_bill) AS mean_bill
FROM tips WHERE SIZE >= 3 
GROUP BY smoker, sex 
HAVING count(*) > 10 AND avg(tip) > 3 
ORDER BY mean_bill DESC 
LIMIT 2
"""

smoker  sex 
Yes     Male    29.576111
No      Male    25.991750
Name: total_bill, dtype: float64

In [5]:
# multiple features, multiple agg functions => dataframe, but column index is multiindex    
(
tips # from
.groupby(['sex','day','smoker']) 
[['tip','total_bill']].agg(['mean','median']) # select
.head(5) # limit 
)
#                        tip        total_bill        
#                       mean median       mean  median
# sex  day  smoker                                    
# Male Thur Yes     3.058000  2.780  19.171000  17.645
#           No      2.941500  2.405  18.486500  16.975
#      Fri  Yes     2.741250  2.600  20.452500  17.215
#           No      2.500000  2.500  17.475000  17.475
#      Sat  Yes     2.879259  3.000  21.837778  20.290

q = """SELECT sex, DAY, smoker, avg(tip) tip_mean, max(tip) tip_max, avg(total_bill) t_bill_mean, max(total_bill) t_bill_max FROM tips GROUP BY sex, DAY, smoker"""
print(duckdb.query(q).df()[:5])
#       sex  day smoker  tip_mean  tip_max  t_bill_mean  t_bill_max
# 0  Female  Sun     No  3.329286     5.20    20.824286       35.26
# 1    Male  Sun     No  3.115349     6.00    20.403256       48.17
# 2    Male  Sat     No  3.256563     9.00    19.929063       48.33
# 3  Female  Sat     No  2.724615     4.67    19.003846       35.83
# 4    Male  Sat    Yes  2.879259    10.00    21.837778       50.81

tip        total_bill        
                      mean median       mean  median
sex  day  smoker                                    
Male Thur Yes     3.058000  2.780  19.171000  17.645
          No      2.941500  2.405  18.486500  16.975
     Fri  Yes     2.741250  2.600  20.452500  17.215
          No      2.500000  2.500  17.475000  17.475
     Sat  Yes     2.879259  3.000  21.837778  20.290

      sex  day smoker  tip_mean  tip_max  t_bill_mean  t_bill_max
0  Female  Sun     No  3.329286     5.20    20.824286       35.26
1    Male  Sun     No  3.115349     6.00    20.403256       48.17
2    Male  Sat     No  3.256563     9.00    19.929063       48.33
3  Female  Sat     No  2.724615     4.67    19.003846       35.83
4    Male  Sat    Yes  2.879259    10.00    21.837778       50.81


In [6]:
# pandas
(
tips # from
.query("day in ['Sun', 'Sat', 'Thur']") # where
.groupby(['sex', 'smoker']) 
.filter(lambda g: len(g)>=28 and g.size.mean()>=2) # having
.groupby(['sex', 'smoker'], observed=True) 
[['total_bill','tip']].agg({'total_bill':['count', 'mean'], 'tip':'mean'}) # select
# .agg(['count', 'mean'])
)

# tips.query("day in ['Sun','Sat','Thur']").groupby(['sex', 'smoker']).filter(lambda df: (len(df)>=28) & (df['size'].mean()>=2)).groupby(['sex', 'smoker'])[['total_bill','tip']].agg(['count','mean'])

# SQL 
q = """
SELECT sex, smoker, count(*), avg(total_bill), avg(tip) FROM tips 
WHERE DAY IN ('Sun','Sat','Thur') 
GROUP BY sex, smoker
HAVING count(*)>=28 AND avg(size)>=2
"""
duckdb.query(q).df()


total_bill                  tip
                   count       mean      mean
sex    smoker                                
Female No             52  18.056731  2.760000
Male   No             95  19.840000  3.126316
       Yes            52  22.566346  3.098846

,sex,smoker,count_star(),avg(total_bill),avg(tip)
0,Female,No,52,18.056731,2.760000
1,Male,No,95,19.840000,3.126316
2,Male,Yes,52,22.566346,3.098846


In [7]:
# value_counts with Where 
# M1: use apply 
def f(df, n):
    return pd.Series({'size': len(df), 'pct': len(df)/n})
(temp := tips.query("tip > 2.0")).groupby('time').apply(f, n=len(temp))

# M2: value_counts
# pd.concat([(temp := tips.query("tip > 2.0")).time.value_counts(), temp.time.value_counts(normalize=True)], axis=1)
pd.concat([(temp := tips.query("tip > 2.0")).value_counts(['time']), temp.value_counts(['time'], normalize=True)], axis=1, keys=['size','pct'])

# M3: self-defined function
tips.query("tip>2.0").time.pipe(value_counts_plus)


q=""" SELECT time, count(*), 1.0*count(*)/(SELECT count(*) FROM tips where tip>2.0) AS rate FROM tips where tip>2.0 GROUP BY time """
duckdb.query(q).df()





,size,pct
time,,
Lunch,39.0,0.23494
Dinner,127.0,0.76506


,size,pct
time,,
Dinner,127,0.76506
Lunch,39,0.23494


stats,count,rate
values,,
Dinner,127,0.76506
Lunch,39,0.23494


,time,count_star(),rate
0,Dinner,127,0.76506
1,Lunch,39,0.23494


In [8]:
# show the distinct values of a feature
# M1
tips.day.unique()
# M2
tips.day.drop_duplicates()

q = """SELECT distinct DAY FROM tips"""
duckdb.query(q).df()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

0      Sun
19     Sat
77    Thur
90     Fri
Name: day, dtype: category
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

,day
0,Sun
1,Sat
2,Thur
3,Fri


In [9]:
tips[['time','day']].drop_duplicates()

q = """SELECT DISTINCT time, DAY FROM tips"""
duckdb.query(q).df()


,time,day
0,Dinner,Sun
19,Dinner,Sat
77,Lunch,Thur
90,Dinner,Fri
220,Lunch,Fri
243,Dinner,Thur


,time,day
0,Dinner,Sun
1,Dinner,Sat
2,Lunch,Thur
3,Dinner,Fri
4,Lunch,Fri
5,Dinner,Thur


In [10]:
tips.groupby(['time','day']).size().sort_index(ascending=False)
# tips.value_counts(['time','day']).sort_index(ascending=False)
# tips[['time','day']].value_counts().sort_index(ascending=False)

q = """SELECT time, DAY, count(*) FROM tips GROUP BY time, DAY ORDER BY time DESC """
duckdb.query(q).df()

time    day 
Dinner  Sun     76
        Sat     87
        Fri     12
        Thur     1
Lunch   Sun      0
        Sat      0
        Fri      7
        Thur    61
dtype: int64

,time,day,count_star()
0,Dinner,Sun,76
1,Dinner,Sat,87
2,Dinner,Fri,12
3,Dinner,Thur,1
4,Lunch,Thur,61
5,Lunch,Fri,7


In [11]:
# find top total_bills by time and day 

# M1
# tips.query("sex=='Male'").groupby(['time','day'])[['total_bill', 'tip']].agg(['min','max','mean']).sort_index(level=[0,1], ascending=[False,False]) # we see the order of execution: from, where, group by, select, order by 

# M2
(
tips
.query("sex == 'Male'")
.groupby(gp_cols := ["time",'day'])
.filter(lambda g: g.tip.max()>=5)
.groupby(gp_cols, observed=True)
[['total_bill','tip']].agg(['min','max','mean'])
.sort_index(level=gp_cols, ascending=[False, False])
)

q = """
SELECT time, DAY, min(total_bill), max(total_bill), avg(total_bill), min(tip), max(tip), avg(tip) 
FROM tips
WHERE sex = 'Male'
GROUP BY time, DAY
ORDER BY time, DAY 
"""
duckdb.query(q).df()

total_bill                     tip                
                   min    max       mean   min   max      mean
time   day                                                    
Dinner Sun        7.25  48.17  21.887241  1.32   6.5  3.220345
       Sat        7.74  50.81  20.802542  1.00  10.0  3.083898
Lunch  Thur       7.51  41.19  18.714667  1.44   6.7  2.980333

,time,day,min(total_bill),max(total_bill),avg(total_bill),min(tip),max(tip),avg(tip)
0,Lunch,Thur,7.51,41.19,18.714667,1.44,6.70,2.980333
1,Lunch,Fri,8.58,13.42,11.386667,1.58,2.20,1.900000
2,Dinner,Fri,12.03,40.17,23.487143,1.50,4.73,3.032857
3,Dinner,Sat,7.74,50.81,20.802542,1.00,10.00,3.083898
4,Dinner,Sun,7.25,48.17,21.887241,1.32,6.50,3.220345


In [12]:
# tips[['total_bill', 'tip']].gt()

tips.query("total_bill > total_bill.mean() or tip > tip.mean()")

,total_bill,tip,sex,smoker,day,time,size
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
7,26.88,3.12,Male,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
238,35.83,4.67,Female,No,Sat,Dinner,3
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2


In [13]:
# select all records whose (total_bill,tip) > avg(total_bill, tip), then find their total_bill max, group by day, and show only day with group size > 10
# M1
(
tips # from
.query(" or ".join(f"{var} > {var}.mean()" for var in ["total_bill", "tip"])) # where
.groupby('day') 
.filter(lambda g: len(g) > 10) # having
.groupby('day', observed=True)
.total_bill.max() # select 
)

# M2
# tips[tips[['total_bill', 'tip']].gt(tips[['total_bill', 'tip']].mean()).any(axis=1)] # M2 


q = """
SELECT DAY, max(total_bill)
FROM tips t1
WHERE tip > (SELECT avg(tip) FROM tips) OR total_bill > (SELECT avg(total_bill) FROM tips)
GROUP BY DAY
HAVING count(*) > 10
"""
duckdb.query(q).df()



day
Sun     48.17
Sat     50.81
Thur    43.11
Name: total_bill, dtype: float64

,day,max(total_bill)
0,Sun,48.17
1,Sat,50.81
2,Thur,43.11


In [14]:
# for each group, find those who have total_bill or tip higher than respective mean
cols = ['total_bill','tip']

# M1: use transform. Not similar to SQL syntax. First, make two new columns. 
means = tips.groupby(['sex', 'smoker'])[cols].transform('mean')
filt = tips[cols].gt(means).any(axis=1)
tips[filt]

# M2: use apply. similar to SQL
(
tips # from 
.groupby(['sex','smoker'], group_keys=False)
.apply(lambda g: g[g[cols].gt(g[cols].mean()).any(axis=1)]) 
.sort_index()
)

# SQL:
q = """
SELECT * 
FROM tips JOIN (
	SELECT sex, smoker, avg(total_bill) AS avg_total_bill, avg(tip) AS avg_tip FROM tips GROUP BY sex, smoker
    ) TEMP 
    using (sex, smoker)
WHERE total_bill > avg_total_bill OR tip > avg_tip 
"""

# -- use window function instead
# select * FROM (
# select *, 
# avg(total_bill) over (partition by sex, smoker) AS avg_total_bill, 
# avg(tip) over (partition by sex, smoker) AS avg_tip
# from tips) TEMP
# WHERE total_bill > avg_total_bill OR tip > avg_tip


,total_bill,tip,sex,smoker,day,time,size
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
7,26.88,3.12,Male,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
238,35.83,4.67,Female,No,Sat,Dinner,3
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2


,total_bill,tip,sex,smoker,day,time,size
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4
7,26.88,3.12,Male,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
238,35.83,4.67,Female,No,Sat,Dinner,3
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2


In [15]:
# make a new column as the tip_rate
tips.assign(tips_rate=tips.tip / tips.total_bill)

# sql
q = """SELECT *, tip/total_bill AS tip_rate FROM tips;"""
duckdb.query(q).df()

,total_bill,tip,sex,smoker,day,time,size,tips_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


,total_bill,tip,sex,smoker,day,time,size,tip_rate
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


In [81]:
tips.groupby('day').agg({'tip':[('avg_tip','mean')], 'day':'count'})

q="""SELECT DAY, avg(tip) AS avg_tip, count(day) FROM tips GROUP BY DAY"""
duckdb.query(q).df()


,tip,day
,avg_tip,count
day,,
Thur,2.771452,62
Fri,2.734737,19
Sat,2.993103,87
Sun,3.255132,76


,day,avg_tip,"count(""day"")"
0,Sun,3.255132,76
1,Sat,2.993103,87
2,Thur,2.771452,62
3,Fri,2.734737,19


In [95]:
# show top 5 records with largest total_bill
tips.nlargest(5, columns='total_bill')

q = """SELECT * FROM tips ORDER BY total_bill DESC LIMIT 5"""
duckdb.query(q).df()

,total_bill,tip,sex,smoker,day,time,size
170,50.81,10.00,Male,Yes,Sat,Dinner,3
212,48.33,9.00,Male,No,Sat,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
156,48.17,5.00,Male,No,Sun,Dinner,6
182,45.35,3.50,Male,Yes,Sun,Dinner,3


,total_bill,tip,sex,smoker,day,time,size
0,50.81,10.00,Male,Yes,Sat,Dinner,3
1,48.33,9.00,Male,No,Sat,Dinner,4
2,48.27,6.73,Male,No,Sat,Dinner,4
3,48.17,5.00,Male,No,Sun,Dinner,6
4,45.35,3.50,Male,Yes,Sun,Dinner,3


In [165]:
# find all males who have tip higher than males' average

# M0: verbose
tips[(tips.sex=="Male") & (tips.tip.gt(tips.query("sex=='Male'").tip.mean()))][:1]

# M1
tips.query("sex=='Male'").query("tip > tip.mean()")[:1]

# M2
male_mean = tips.query("sex=='Male'").tip.mean()
tips.query("sex=='Male' and tip>@male_mean")[:1]

# M3: apply. inefficient, because also calculate female group 
tips.groupby('sex').apply(lambda g: g.query("tip>tip.mean()")).loc['Male'][:1]

# M4: transform
tips.assign(means=tips.groupby('sex').tip.transform('mean')).query("sex=='Male' and tip > means")[:1]

q ="""
SELECT * FROM tips WHERE sex = 'Male' AND tip > (SELECT avg(tip) FROM tips WHERE sex='Male')
"""

q = """
SELECT * FROM tips WHERE sex = 'Male' AND tip > (SELECT avg(tip) FROM tips WHERE sex='Male')
"""


,total_bill,tip,sex,smoker,day,time,size
2,21.01,3.5,Male,No,Sun,Dinner,3


,total_bill,tip,sex,smoker,day,time,size
2,21.01,3.5,Male,No,Sun,Dinner,3


,total_bill,tip,sex,smoker,day,time,size
2,21.01,3.5,Male,No,Sun,Dinner,3


,total_bill,tip,sex,smoker,day,time,size
2,21.01,3.5,Male,No,Sun,Dinner,3


,total_bill,tip,sex,smoker,day,time,size,means
2,21.01,3.5,Male,No,Sun,Dinner,3,3.089618


In [93]:
# find all males who give higher tip than the max tip in female

# M1
max_tip_female = tips.query("sex == 'Female'").tip.max() # subquery 
tips.query("sex == 'Male' and tip > @max_tip_female")[['tip']].agg(['mean','count']) # or .agg({'tip':['mean', 'count']})

# M2
tips[ (tips.sex =='Male') & (tips.tip > tips.query("sex == 'Female'").tip.max())][['tip']].agg(['mean','count']) # or .agg({'tip':['mean', 'count']})

q="""SELECT avg(tip), count(tip) FROM tips WHERE sex = 'Male' AND tip > (SELECT max(tip) FROM tips WHERE sex='Female')"""
duckdb.query(q).df()

,tip
mean,8.002
count,5.000


,tip
mean,8.002
count,5.000


,avg(tip),count(tip)
0,8.002,5


In [110]:
# find average tip for cohort with total size variable > 100 
(
tips # from 
.groupby(['sex','smoker']) 
.filter(lambda g: g['size'].sum()>100) # remember that we can't use `g.size` here!!! This gives us the total number of entries in g, not the feature!!!!!!
.groupby(['sex','smoker'], observed=True)
.tip.agg(['mean','size'])
)

q = """SELECT sex, smoker, avg(tip) FROM tips GROUP BY sex, smoker HAVING sum(size)>100"""
duckdb.query(q).df().set_index(['sex','smoker'])


mean  size
sex    smoker                
Female No      2.773519    54
Male   No      3.113402    97
       Yes     3.051167    60

avg(tip)
sex    smoker          
Female No      2.773519
Male   No      3.113402
       Yes     3.051167

In [193]:
# find cohort size with size >= some threshold 
# https://stackoverflow.com/questions/27718650/sql-select-group-by-a-having-count1-1-equivalent-in-python-pandas
# if size > 1, then we are to show how many duplicate rows! (think of cohort kindof like an id, e.g., email, studend id)
threshold=20
cohort = ['sex','smoker','day']

# SQL style 
(
tips
.groupby(cohort)
.filter(lambda g: len(g)>(threshold))
.groupby(cohort, observed=True)
.size()
)

# pandas way 
(
tips
.groupby(cohort)
.size()
[lambda v: v>threshold] # remember that before callable selection, it's still a series
)

q="""SELECT sex, smoker, DAY, count(*) FROM tips GROUP BY sex, smoker, DAY HAVING count(*)>20"""
duckdb.query(q).df()

sex     smoker  day 
Male    No      Sun     43
                Sat     32
        Yes     Sat     27
Female  No      Thur    25
dtype: int64

sex     smoker  day 
Male    Yes     Sat     27
        No      Sat     32
                Sun     43
Female  No      Thur    25
dtype: int64

,sex,smoker,day,count_star()
0,Male,No,Sun,43
1,Male,No,Sat,32
2,Male,Yes,Sat,27
3,Female,No,Thur,25


,total_bill,tip,sex,smoker,day,time,size
236,12.60,1.00,Male,Yes,Sat,Dinner,2
237,32.83,1.17,Male,Yes,Sat,Dinner,2
135,8.51,1.25,Female,No,Thur,Lunch,2
75,10.51,1.25,Male,No,Sat,Dinner,2
235,10.07,1.25,Male,No,Sat,Dinner,2
...,...,...,...,...,...,...,...
47,32.40,6.00,Male,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
23,39.42,7.58,Male,No,Sat,Dinner,4
212,48.33,9.00,Male,No,Sat,Dinner,4


In [194]:
cohort = ['sex','smoker','day']

# variation of the previous: get those rows whose cohort size (a cohort is sex, smoker, day) > threshold
tips.groupby(cohort).filter(lambda g: len(g)>(threshold)).sort_values('tip')

# M1: use window function
q = """
SELECT * FROM (SELECT *, count(*) OVER (PARTITION BY sex, smoker, day) FROM tips) TEMP WHERE count>20 ORDER BY tip
"""

# M2: calculate the group with size >20 first, then do natural inner join 
q = """
WITH t1 AS (SELECT sex, smoker, DAY, count(*) FROM tips GROUP BY sex, smoker, DAY HAVING count(*)>20)
SELECT * FROM tips t2 NATURAL JOIN t1 ORDER BY tip
"""

# M3: like M2, but more verbose 
q = """SELECT t2.tip, t2.sex, t2.smoker, t2.DAY FROM 
(SELECT sex, smoker, DAY, count(*) FROM tips GROUP BY sex, smoker, DAY HAVING count(*)>20) t1, tips t2
WHERE t1.sex=t2.sex AND t1.smoker=t2.smoker AND t1.DAY = t2.DAY order by t2.total_bill"""
duckdb.query(q).df()

,total_bill,tip,sex,smoker,day,time,size
236,12.60,1.00,Male,Yes,Sat,Dinner,2
237,32.83,1.17,Male,Yes,Sat,Dinner,2
135,8.51,1.25,Female,No,Thur,Lunch,2
75,10.51,1.25,Male,No,Sat,Dinner,2
235,10.07,1.25,Male,No,Sat,Dinner,2
...,...,...,...,...,...,...,...
47,32.40,6.00,Male,No,Sun,Dinner,4
59,48.27,6.73,Male,No,Sat,Dinner,4
23,39.42,7.58,Male,No,Sat,Dinner,4
212,48.33,9.00,Male,No,Sat,Dinner,4


In [ ]:
# find number of unique rows 
cohort = ['sex','smoker','day']
tips.drop_duplicates().shape[0]

q = """
SELECT count(*) FROM (SELECT DISTINCT * FROM tips) temp
"""

In [216]:
# find all distinct combinations of several columns
cohort = ['sex','smoker','day']
tips[cohort].drop_duplicates() # tips.groupby(['sex','smoker','day']).size().index
tips.groupby(cohort).ngroups # this show the number only 

# q = """SELECT DISTINCT sex, smoker, DAY FROM tips"""
# duckdb.query(q).df()

,sex,smoker,day
0,Female,No,Sun
1,Male,No,Sun
19,Male,No,Sat
21,Female,No,Sat
56,Male,Yes,Sat
67,Female,Yes,Sat
77,Male,No,Thur
80,Male,Yes,Thur
82,Female,No,Thur
90,Male,Yes,Fri


16

## tips dataset: top tipper problems
- find ALL top tipper (also by cohort)
- find top-k tipper (also by cohort)
- find second tipper
- cohort = sex, smoker

In [23]:
# find ALL top tipper by tip rate (with largest tip:total_bill ratio)

# M1: show the tip_rate as well
tips.assign(tip_rate=tips.tip.div(tips.total_bill)).query("tip_rate == tip_rate.max()")
# M2: do not show tip_rate
tips[(temp := tips.tip.div(tips.total_bill)).eq(temp.max())]

q = """
WITH TEMP AS (SELECT *, tip/total_bill AS tip_rate FROM tips)
SELECT * FROM TEMP 
WHERE tip_rate = (SELECT max(tip_rate) FROM TEMP)
"""

# this is wrong:
# SELECT * FROM (SELECT *, tip/total_bill AS tip_rate FROM tips) TEMP 
# WHERE tip_rate = (SELECT max(tip_rate) FROM TEMP)

,total_bill,tip,sex,smoker,day,time,size,tip_rate
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


,total_bill,tip,sex,smoker,day,time,size
172,7.25,5.15,Male,Yes,Sun,Dinner,2


In [29]:
# find top tipper by cohort

# M1: check which one has tip == tip.max()
# tips.groupby(['sex','smoker']).apply(lambda g: g[g.tip == g.tip.max()])
# tips.groupby(['sex','smoker']).apply(lambda g: g.query("tip == tip.max()"))

# M2: use nlargest directly
tips.groupby(['sex','smoker']).apply(lambda g: g.nlargest(n=1, columns='tip', keep='all'))

q = """
SELECT * FROM tips t1 
WHERE t1.tip = (SELECT max(tip) FROM tips t2 WHERE t1.sex = t2.sex AND t1.smoker = t2.smoker) 
ORDER BY tip desc
"""

total_bill   tip     sex smoker  day    time  size
sex    smoker                                                        
Male   Yes    170       50.81  10.0    Male    Yes  Sat  Dinner     3
       No     212       48.33   9.0    Male     No  Sat  Dinner     4
Female Yes    214       28.17   6.5  Female    Yes  Sat  Dinner     3
       No     52        34.81   5.2  Female     No  Sun  Dinner     4

In [80]:
# find top-k tipper by cohort
k = 3

# M1
# tips.groupby(['sex','smoker']).apply(lambda g: g.nlargest(n=k, columns='tip', keep='all'))

# M2 
tips.groupby(['sex','smoker']).apply(lambda g: g.sort_values('tip', ascending=False).head(k)).sort_values('tip', ascending=False) # this method can't handle duplicate case!


# M1
q = """
WITH TEMP AS (SELECT *, RANK() OVER (PARTITION BY sex, smoker ORDER BY tip DESC) as rank FROM tips)
SELECT * FROM TEMP WHERE RANK <= 3
ORDER BY tip DESC
"""
duckdb.query(q).df()


# M2
q = """
SELECT * FROM tips t1 
WHERE (SELECT count(*) FROM tips t2 WHERE t1.tip < t2.tip AND t1.sex = t2.sex AND t1.smoker = t2.smoker) < 3
ORDER BY tip DESC 
"""
# duckdb.query(q).df()

# M3
# don't use distinct!
q = """
SELECT *, count(*) OVER() FROM tips t1 WHERE tip IN
(SELECT tip FROM tips t2 WHERE t1.sex=t2.sex AND t1.smoker=t2.smoker ORDER BY tip DESC LIMIT 3)
ORDER BY tip DESC
"""
# duckdb.query(q).df()

total_bill    tip     sex smoker   day    time  size  \
sex    smoker                                                             
Male   Yes    170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       No     212       48.33   9.00    Male     No   Sat  Dinner     4   
              23        39.42   7.58    Male     No   Sat  Dinner     4   
              59        48.27   6.73    Male     No   Sat  Dinner     4   
       Yes    183       23.17   6.50    Male    Yes   Sun  Dinner     4   
Female Yes    214       28.17   6.50  Female    Yes   Sat  Dinner     3   
Male   Yes    181       23.33   5.65    Male    Yes   Sun  Dinner     2   
Female No     52        34.81   5.20  Female     No   Sun  Dinner     4   
              85        34.83   5.17  Female     No  Thur   Lunch     4   
              155       29.85   5.14  Female     No   Sun  Dinner     5   
       Yes    197       43.11   5.00  Female    Yes  Thur   Lunch     4   
              73        25.28   5.00  Female    Yes   Sat  Dinner     2   

                    avg_tip  avg_total_bill  
sex    smoker                                
Male   Yes    170  3.051167       22.284500  
       No     212  3.113402       19.791237  
              23   3.113402       19.791237  
              59   3.113402       19.791237  
       Yes    183  3.051167       22.284500  
Female Yes    214  2.931515       17.977879  
Male   Yes    181  3.051167       22.284500  
Female No     52   2.773519       18.105185  
              85   2.773519       18.105185  
              155  2.773519       18.105185  
       Yes    197  2.931515       17.977879  
              73   2.931515       17.977879

,total_bill,tip,sex,smoker,day,time,size,avg_tip,avg_total_bill,rank
0,50.81,10.00,Male,Yes,Sat,Dinner,3,3.051167,22.284500,1
1,48.33,9.00,Male,No,Sat,Dinner,4,3.113402,19.791237,1
2,39.42,7.58,Male,No,Sat,Dinner,4,3.113402,19.791237,2
3,48.27,6.73,Male,No,Sat,Dinner,4,3.113402,19.791237,3
4,23.17,6.50,Male,Yes,Sun,Dinner,4,3.051167,22.284500,2
5,28.17,6.50,Female,Yes,Sat,Dinner,3,2.931515,17.977879,1
6,23.33,5.65,Male,Yes,Sun,Dinner,2,3.051167,22.284500,3
7,34.81,5.20,Female,No,Sun,Dinner,4,2.773519,18.105185,1
8,34.83,5.17,Female,No,Thur,Lunch,4,2.773519,18.105185,2
9,29.85,5.14,Female,No,Sun,Dinner,5,2.773519,18.105185,3


# students, enrollments, courses dataset

# titanic dataset

In [234]:
titanic = sns.load_dataset('titanic')
cohort = ['sex','class', 'alone']
titanic.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'], dtype=object)

In [247]:
titanic.groupby(cohort).survived.mean().loc[('female','Second',True)]

0.90625

In [239]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [219]:
# find the survival rate for each cohort 
titanic.groupby(cohort).survived.mean().sort_values()
# sns.barplot(data=titanic, x='class', y='survived', hue='sex')

q = """SELECT sex, CLASS, avg(survived) FROM titanic GROUP BY sex, CLASS ORDER BY avg(survived)"""
duckdb.query(q).df()

sex     class 
male    Third     0.135447
        Second    0.157407
        First     0.368852
female  Third     0.500000
        Second    0.921053
        First     0.968085
Name: survived, dtype: float64

,sex,class,avg(survived)
0,male,Third,0.135447
1,male,Second,0.157407
2,male,First,0.368852
3,female,Third,0.500000
4,female,Second,0.921053
5,female,First,0.968085


In [75]:
# find employee who earns more than their manager
employee = read_from_db("select * from employee", db_name='postgre_db')
employee.merge(employee, left_on='managerid', right_on='id').query("salary_x > salary_y")

q ="""SELECT * FROM employee e1, employee e2 WHERE e1.managerid = e2.id AND e1.salary > e2.salary """
duckdb.query(q).df()

,id_x,name_x,salary_x,managerid_x,id_y,name_y,salary_y,managerid_y
0,1,Joe,70000.0,3.0,3,Sam,60000.0,NaN


,id,name,salary,managerid,id_2,name_2,salary_2,managerid_2
0,1,Joe,70000.0,3.0,3,Sam,60000.0,NaN


In [76]:
tips.groupby(['sex','day']).tip.median().sort_index(level=[0,1], ascending=[False,True])

sex     day 
Female  Thur    2.005
        Fri     3.000
        Sat     2.625
        Sun     3.500
Male    Thur    2.530
        Fri     2.600
        Sat     3.000
        Sun     3.085
Name: tip, dtype: float64

In [77]:
temp = pd.DataFrame({'name':['Zfdsfs','Areew','Bfds','Cfds', 'Female', 'Male', 'Female', 'Female', 'Male']})
temp.name.sort_values()

1     Areew
2      Bfds
3      Cfds
4    Female
6    Female
7    Female
5      Male
8      Male
0    Zfdsfs
Name: name, dtype: object

In [78]:
tips.smoker.sort_values()



67     Yes
176    Yes
76     Yes
177    Yes
178    Yes
      ... 
89      No
91      No
94      No
78      No
243     No
Name: smoker, Length: 244, dtype: category
Categories (2, object): ['Yes', 'No']